In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from Python.pandas.feature_selection import *

In [ ]:
data = pd.read_csv("./data/2_train.csv", sep=';', index_col='id')
data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
id,,,,,,,,,,,,,,,,,,,,,
1,-,housemaid,married,basic.4y,.,no,?,telephone,may,NaN,...,-,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,57,services,married,high.school,NaN,no,no,telephone,may,.,...,1,999,0,?,1.1,93.994,-36.4,4.857,5191.0,no
3,37,services,NaN,high.school,no,?,no,telephone,may,NaN,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,NaN,1.1,93.994,-36.4,4.857,5191.0,no
5,56,services,married,high.school,no,no,yes,?,may,mon,...,1,999,?,NaN,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41184,NaN,retired,NaN,NaN,no,NaN,NaN,cellular,nov,fri,...,1,NaN,?,-,-1.1,94.767,-50.8,1.028,4963.6,yes
41185,46,blue-collar,NaN,professional.course,no,no,no,cellular,nov,fri,...,1,.,-,?,-1.1,94.767,-50.8,1.028,4963.6,no
41186,56,retired,.,-,no,yes,no,cellular,nov,fri,...,2,.,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no


In [ ]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base#missingpy包辅助项
from missingpy import MissForest

imbImput=MissForest(n_estimators=10,min_samples_leaf=4000)                                   #缺失值填补
residMiss=imbImput.fit_transform(data.iloc[:,[*range(1,21)]])
dataResid=pd.DataFrame(residMiss,columns=data.iloc[:,[*range(1,21)]].columns)
dataResid

ValueError: could not convert string to float: 'housemaid'

In [ ]:
print("数据形状：", data.shape)
print("列名:", data.columns)
data.head(5)

In [ ]:
object_feature_helper(data)

In [ ]:
data_age = data['age'].apply(lambda age: age_to_int(age))
data_age.value_counts().sort_index().plot(kind='bar')

In [ ]:
job_mapping = {'admin.': 0, 'blue-collar': 1, 'technician': 2, 'services': 3, '-': 99, 'management': 0, '.': 99,
               '?': 99, 'retired': 11, 'entrepreneur': 0, 'self-employed': 10, 'unemployed': 11, 'housemaid': 10,
               'student': 11}
data_job, job_ohe = one_hot_encoder_column(data, 'job', job_mapping, fill_na=99)

In [ ]:
marital_mapping = {'married': 0, 'single': 1, 'divorced': 1, '-': 1, '.': 1, '?': 1}
data_marital = data['marital'].apply(lambda marital: 0 if marital == 'married' else 1)

data_marital.value_counts().sort_index().plot(kind='bar')

In [ ]:
education_mapping = {'university.degree': 2, 'high.school': 1, 'basic.9y': 0, 'professional.course': 3, 'basic.4y': 0, '-': 0, '.': 0, 'basic.6y': 0, '?': 0, 'illiterate': 0, 0: 0}
data['education'] = data['education'].fillna(0)
data_education = data['education'].apply(lambda edu: education_mapping[edu])
data_education.value_counts().sort_index().plot(kind='bar')


In [ ]:
housing_mapping = {'yes': 0, 'no': 1, '-': 0, '.': 0, '?': 0, 0:0}
data_housing, housing_ohe = one_hot_encoder_column(data, 'housing', housing_mapping, fill_na=99)
data_housing = pd.DataFrame(data_housing)

In [ ]:
contact_mapping = {'cellular': 0, 'telephone': 1, '-': 2, '.': 2, '?': 2}
data_contact, contact_ohe = one_hot_encoder_column(data, 'contact', contact_mapping, fill_na=99)
data_contact = pd.DataFrame(data_contact)

In [ ]:
day_of_week_mapping = {'thu': 0, 'mon': 1, 'tue': 2, 'wed': 3, 'fri': 4, '-': 5, '.': 5, '?': 5}
data_day_of_week, day_of_week_mapping_ohe = one_hot_encoder_column(data, 'day_of_week', day_of_week_mapping, fill_na=99)
data_day_of_week = pd.DataFrame(data_day_of_week)

In [ ]:
data_duration = data['duration'].fillna(0)
data_duration[data_duration == '-'] = '0'
data_duration[data_duration == '.'] = '0'
data_duration[data_duration == '?'] = '0'
data_duration = data_duration.astype(int)

In [ ]:
data['campaign'].value_counts()
data_campaign = data['campaign'].fillna(0)
data_campaign[data_campaign == '-'] = '0'
data_campaign[data_campaign == '.'] = '0'
data_campaign[data_campaign == '?'] = '0'
data_campaign = data_campaign.astype(int)

In [ ]:
data['pdays'].value_counts()
non_values = ['-', '.', '?']
data_pdays = data['pdays'].fillna(999)
data_pdays[data_pdays.isin(non_values)] = 999
data_pdays = data_pdays.astype(int)
data_pdays.value_counts()

In [ ]:
data_previous = fill_na(data['previous'], non_list=non_values, na_value=0)
data_previous.value_counts()

In [ ]:
poutcome_mapping = {'nonexistent': 0, 'failure': 1, '-': 0, '.': 0, '?': 0, 'success': 2}
data_poutcome, poutcome_ohe = one_hot_encoder_column(data, 'poutcome', poutcome_mapping, fill_na=0)
data_poutcome = pd.DataFrame(data_poutcome)

data_poutcome

In [ ]:
from sklearn.preprocessing import StandardScaler
data_emp_var_rate = data['emp.var.rate']
data_cons_price_idx = data['cons.price.idx']
data_cons_conf_idx = data['cons.conf.idx']
data_euribor3m = data['euribor3m']
data_nr_employed = data['nr.employed']

In [ ]:
data_y = data['y'].apply(lambda y: 1 if y=='yes' else 0)

In [ ]:
data_all = pd.concat([data_age, data_job, data_marital, data_education, data_housing, data_contact, data_day_of_week,
                      data_duration, data_campaign, data_pdays, data_previous, data_poutcome, data_y], axis=1)

data_all.to_csv("./data/feature_2_test.csv")

